# Wrapping a basic library

We here aim at presenting the interactive wrapping workflow.
For the sake of simplicity, we consider a basic example of *C++* library.

First, import **AutoWIG**.

In [1]:
import autowig

Then, to install and compile the *C++* library we use available **Conda** recipes.

In [2]:
%%bash --err error
conda build -q basic/conda/libbasic -c statiskit
conda install -y -q libbasic --use-local -c statiskit

Removing old build environment
BUILD START: libbasic-1.0.0-0
    (actual version deferred until further download or env creation)
Fetching package metadata ...........
Solving package specifications: ..........
scons: Reading SConscript files ...
scons: done reading SConscript files.
scons: Building targets ...
Install file: "build/cpp/binomial.h" as "/home/main/miniconda/envs/_build/include/basic/binomial.h"
Install file: "build/cpp/overload.h" as "/home/main/miniconda/envs/_build/include/basic/overload.h"
g++ -o build/cpp/overload.os -c -x c++ -std=c++0x -Wwrite-strings -fPIC -DBOOST_PYTHON_DYNAMIC_LIB -I/home/main/miniconda/envs/_build/include -I/home/main/miniconda/envs/_build/include/python2.7 build/cpp/overload.cpp
g++ -o build/cpp/binomial.os -c -x c++ -std=c++0x -Wwrite-strings -fPIC -DBOOST_PYTHON_DYNAMIC_LIB -I/home/main/miniconda/envs/_build/include -I/home/main/miniconda/envs/_build/include/python2.7 build/cpp/binomial.cpp
g++ -o /home/main/miniconda/envs/_build/lib/libbasi

Once the headers have been installed in the system, we parse them with relevant compilation flags.

In [3]:
import sys
asg = autowig.AbstractSemanticGraph()
asg = autowig.parser(asg, [sys.prefix + '/include/basic/overload.h',
                           sys.prefix + '/include/basic/binomial.h'],
                          ['-x', 'c++', '-std=c++11'],
                          silent = True)

Since most of **AutoWIG** guidelines are respected, the `default` `controller` implementation is suitable.

In [4]:
autowig.controller.plugin = 'default'
asg = autowig.controller(asg)

In order to wrap the library we need to select the `boost_python_internal` `generator` implementation.

In [5]:
autowig.generator.plugin = 'boost_python_internal'
wrappers = autowig.generator(asg,
                             module = 'basic/src/py/_module.cpp',
                             decorator = 'basic/src/py/basic/_module.py',
                             prefix = 'wrapper_')

The wrappers are only generated in-memory.
It is therefore needed to write them on the disk to complete the process.

In [6]:
wrappers.write()

Here is an example of the generated wrappers.
We here present the wrappers for the `BinomialDistribution` class.

In [7]:
asg['class ::BinomialDistribution'].boost_python_export

/home/main/AutoWIG/doc/examples/basic/src/py/wrapper_4046a8421fe9587c9dfbc97778162c7d.cpp

Once the wrappers are written on disk, we need to compile and install the *Python* bindings.

In [8]:
%%bash --err error
conda build -q basic/conda/python-basic -c statiskit
conda install -y -q python-basic --use-local -c statiskit

Removing old build environment
BUILD START: python-basic-1.0.0-0
    (actual version deferred until further download or env creation)
Fetching package metadata ...........
Solving package specifications: ..........
scons: Reading SConscript files ...
scons: done reading SConscript files.
scons: Building targets ...
g++ -o build/py/_module.h.gch -x c++-header -c -fPIC -x c++ -std=c++0x -Wwrite-strings -ftemplate-depth-100 -DBOOST_PYTHON_DYNAMIC_LIB -I/home/main/miniconda/envs/_build/include -I/home/main/miniconda/envs/_build/include/python2.7 build/py/_module.h
Install file: "build/cpp/binomial.h" as "/home/main/miniconda/envs/_build/include/basic/binomial.h"
Install file: "build/cpp/overload.h" as "/home/main/miniconda/envs/_build/include/basic/overload.h"
g++ -o build/cpp/binomial.os -c -x c++ -std=c++0x -Wwrite-strings -fPIC -DBOOST_PYTHON_DYNAMIC_LIB -I/home/main/miniconda/envs/_build/include -I/home/main/miniconda/envs/_build/include/python2.7 build/cpp/binomial.cpp
g++ -o build/cp

Finally, we can hereafter use the *C++* library in the *Python* interpreter.

In [9]:
import basic
binomial = basic.BinomialDistribution(1, .5)
binomial

In [10]:
binomial.pmf(0)

0.5

In [11]:
binomial.pmf(1)

0.5

In [12]:
binomial.n = 0
binomial

In [13]:
binomial.pmf(0)

1.0

In [14]:
try:
    binomial.set_pi(1.1)
except basic.ProbabilityError as error:
    print error.message
else:
    raise Exception('A `basic.ProbabilityError` should have been raise')

a probability must be in the interval [0,1]


/home/main/miniconda/lib/python2.7/site-packages/ipykernel/__main__.py:4: DeprecationWarning: BaseException.message has been deprecated as of Python 2.6
